In [9]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=5
p=floor(Int64,n/u)+1

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)



g=-(x.^2).+(1/u)



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)

randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")



***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=5
Number of equality constraints: l=1


In [10]:
k=2

println("Relaxed order: k=",k)
println("====================")

t=1

println("Sparse order: t=",t)
println("====================")

Relaxed order: k=2
Sparse order: t=1


In [11]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [12]:
using TSSOS


@time begin
    ~,~,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false);
    for j in 1:t-1
        ~,~,data=cs_tssos_higher!(data,TS="block");
    end
end

------------------------------------------------------
The clique sizes of varibles:
[5]
[1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 91              
  Cones                  : 0               
  Scalar variables       : 22              
  Matrix variables       : 8               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00  

(-0.5259681481283173, nothing, TSSOS.mdata_type(5, 0, 6, [2, 2, 2, 2, 2, 2], SparseMatrixCSC{UInt8,UInt32}[
  [1,  1]  =  0x02
  [1,  2]  =  0x01
  [2,  2]  =  0x01
  [1,  3]  =  0x01
  [3,  3]  =  0x01
  [1,  4]  =  0x01
  [4,  4]  =  0x01
  [1,  5]  =  0x01
  [5,  5]  =  0x01
  [2,  6]  =  0x02
  [2,  7]  =  0x01
  [3,  7]  =  0x01
  [2,  8]  =  0x01
  [4,  8]  =  0x01
  [2,  9]  =  0x01
  [5,  9]  =  0x01
  [3, 10]  =  0x02
  [3, 11]  =  0x01
  [4, 11]  =  0x01
  [3, 12]  =  0x01
  [5, 12]  =  0x01
  [4, 13]  =  0x02
  [4, 14]  =  0x01
  [5, 14]  =  0x01
  [5, 15]  =  0x02, 
  [1, 1]  =  0x02, 
  [2, 1]  =  0x02, 
  [3, 1]  =  0x02, 
  [4, 1]  =  0x02, 
  [5, 1]  =  0x02, 
  [5, 1]  =  0x02], Array{Float64,1}[[-0.6088974379888188, -0.2232030019677964, 0.554758433687049, 0.7833688241734378, 0.445805819170801, 0.5519704238273353, 0.6170181962771797, -0.16044819651187492, -0.9526768755181374, 0.9300690508628815, 0.0021934381000776604, 0.766111917723026, 0.39183638876841664, 0.927960971

In [13]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_mix_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[5]
[1]
------------------------------------------------------
  Number of cliques: p=1
  Largest clique size: u=5


  Largest constant trace: a=3.0
  Number of blocks: omega=12
  Number of equality consraints: zeta=162
  Size of the largest block: s^max=16
Modeling time:
  0.558120 seconds (717.99 k allocations: 36.492 MiB)
iter=1   val=-2.451010370239651   gap=2.57367647272079e-16   feas=3.355626257960232
iter=2   val=-0.8170034567465505   gap=0.6080109825999888   feas=0.6620971215894007
iter=4   val=-0.9063256661784256   gap=0.4658003984470651   feas=0.9186556523510263
iter=8   val=-0.6281948583991066   gap=0.08308621838312719   feas=0.4661218968308251
iter=16   val=-0.45523690455644017   gap=0.11268695294573162   feas=0.39949082608570746
iter=32   val=-0.5807222953398592   gap=0.1649246186808473   feas=0.15680917150486903
iter=64   val=-0.5330829597887387   gap=0.1383306693055401   feas=0.12076827595864492
iter=128   val=-0.4980138743004699   gap=0.0961879180737565   feas=0.05740182348588339
iter=256   val=-0.5206550492241858   gap=0.0904167140341381   feas=0.01454147836992249
iter=333   val=-0.5

-0.5173795034129177

In [14]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val=ctpPOP.POP_mix_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    tol=1e-2,use_eqcons_to_get_constant_trace=false)

------------------------------------------------------
The clique sizes of varibles:
[5]
[1]
------------------------------------------------------
  Number of cliques: p=1
  Largest clique size: u=5


  Largest constant trace: a=3.0
  Number of blocks: omega=12
  Number of equality consraints: zeta=162
  Size of the largest block: s^max=16
Modeling time:
  0.516476 seconds (717.95 k allocations: 36.472 MiB)
**LMBM solver:
---------------
| Parameters: |
---------------
n:       162
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.42246092D+00  WK= 0.3716D-01  QK= 0.1858D-01
 NIT=    2  NFE=    2  F= 0.38529892D+00  WK= 0.3716D-01  QK= 0.1858D-01
 NIT=    3  NFE=    3  F= 0.34813691D+00  WK= 0.3716D-01  QK= 0.1858D-01
 NIT=    4  NFE=    4  F= 0.31097490D+00  WK= 0.3716D-01  QK= 0.1858D-01
 NIT=    5  NFE=    5  F= 0.29729605D+00  WK= 0.3716D-01  QK= 0.1858D-01
 NIT=    6  NFE=   50  F= 0.29729605D+00  WK= 0.3716D-01  QK= 0.1858D-01
 NIT=    7  NFE=   95  F= 0.29729605D+00  WK= 0.3716D-01  QK= 0.1858D-01
 NIT=    8  NFE=  140  F= 0.29729605D+00  WK= 0.3716D-01  QK= 0.1858D-01
 NIT=    9  NFE=  185  F= 0.29729605D+00  WK

-1.724835758920147

In [15]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        1
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

0.0